In [ ]:

!pip install git+https://github.com/openai/whisper.git

!pip install tqdm

!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

## Setup

We used Python 3.9.9 and [PyTorch](https://pytorch.org/) 1.10.1 to train and test our models, but the codebase is expected to be compatible with Python 3.8-3.11 and recent PyTorch versions. The codebase also depends on a few Python packages, most notably [OpenAI's tiktoken](https://github.com/openai/tiktoken) for their fast tokenizer implementation. You can download and install (or update to) the latest release of Whisper with the following command:

    pip install -U openai-whisper

Alternatively, the following command will pull and install the latest commit from this repository, along with its Python dependencies:

    pip install git+https://github.com/openai/whisper.git 

To update the package to the latest version of this repository, please run:

    pip install --upgrade --no-deps --force-reinstall git+https://github.com/openai/whisper.git

It also requires the command-line tool [`ffmpeg`](https://ffmpeg.org/) to be installed on your system, which is available from most package managers:

```bash
# on Ubuntu or Debian
sudo apt update && sudo apt install ffmpeg

# on Arch Linux
sudo pacman -S ffmpeg

# on MacOS using Homebrew (https://brew.sh/)
brew install ffmpeg

# on Windows using Chocolatey (https://chocolatey.org/)
choco install ffmpeg

# on Windows using Scoop (https://scoop.sh/)
scoop install ffmpeg
```

In [8]:
import os
import whisper
from tqdm import tqdm
import torch
torch.cuda.is_available()
print(whisper.__file__)

c:\Users\W.Mauti\Documents\Anaconda\envs\AI_test\Lib\site-packages\whisper\__init__.py


In [9]:
# Define the folder where the wav files are located
root_folder = "C:/Users/W.Mauti/Documents/Jupiter"
exit_folder = "E:\\SUBRIZI\\Lez 1 02_10"
audio_ext = ".m4a" # .mp3, .wav, .flac ...
language = "it"

## Available models and languages

There are five model sizes, four with English-only versions, offering speed and accuracy tradeoffs. Below are the names of the available models and their approximate memory requirements and inference speed relative to the large model; actual speed may vary depending on many factors including the available hardware.

|  Size  | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
|:------:|:----------:|:------------------:|:------------------:|:-------------:|:--------------:|
|  tiny  |    39 M    |     `tiny.en`      |       `tiny`       |     ~1 GB     |      ~32x      |
|  base  |    74 M    |     `base.en`      |       `base`       |     ~1 GB     |      ~16x      |
| small  |   244 M    |     `small.en`     |      `small`       |     ~2 GB     |      ~6x       |
| medium |   769 M    |    `medium.en`     |      `medium`      |     ~5 GB     |      ~2x       |
| large  |   1550 M   |        N/A         |      `large`       |    ~10 GB     |       1x       |

The `.en` models for English-only applications tend to perform better, especially for the `tiny.en` and `base.en` models. We observed that the difference becomes less significant for the `small.en` and `medium.en` models.

Whisper's performance varies widely depending on the language. The figure below shows a performance breakdown of `large-v3` and `large-v2` models by language, using WERs (word error rates) or CER (character error rates, shown in *Italic*) evaluated on the Common Voice 15 and Fleurs datasets. Additional WER/CER metrics corresponding to the other models and datasets can be found in Appendix D.1, D.2, and D.4 of [the paper](https://arxiv.org/abs/2212.04356), as well as the BLEU (Bilingual Evaluation Understudy) scores for translation in Appendix D.3.

In [10]:
# Set up Whisper client
print("Loading whisper model...")
model = whisper.load_model("medium", device="cuda")
print("Whisper model complete.")

Loading whisper model...
Whisper model complete.


In [11]:
# Get the number of wav files in the root folder and its sub-folders
print("Getting number of files to transcribe...")
num_files = sum(1 for dirpath, dirnames, filenames in os.walk(root_folder) for filename in filenames if filename.endswith(audio_ext))
print("Number of files: ", num_files)


# Transcribe the wav files and display a progress bar
with tqdm(total=num_files, desc="Transcribing Files") as pbar:
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith(audio_ext):
                filepath = str(os.path.join(dirpath, filename))
                result = model.transcribe(filepath, language=language, fp16=False, verbose=False)
                transcription = result['text']
                # Write transcription to text file
                filename_no_ext = os.path.splitext(filename)[0]
                with open(os.path.join(exit_folder, filename_no_ext + '.txt'), 'w') as f:
                    f.write(transcription)
                pbar.update(1)


Getting number of files to transcribe...
Number of files:  1


Transcribing Files: 100%|██████████| 1/1 [47:55<00:00, 2875.42s/it]
